In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

In [2]:

# Database Configuration
DB_CONFIG = {
    'host': 'localhost',
    'database': 'SQLTEST',
    'user': 'postgres',
    'password': 'Admin'
}

In [3]:
# Create a connection string
connection_string = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

In [4]:
# SQL Query to fetch data
query = """
SELECT 
    o.id AS order_id,
    o.groups_carts_id,
    o.status AS order_status,
    o.total_amount,
    o.created_at AS order_created_at,
    o.updated_at AS order_updated_at,
    o.deleted_at AS order_deleted_at,
    o.payment_method,
    o.discount,
    pr.rating,
    pr.comment,
    pn.id AS product_name_id,
    pn.name AS product_name,
    c.id AS category_id,
    c.name AS category_name,
    p.id AS product_id,
    p.vendor_id,
    p.status AS product_status,
    p.created_at AS product_created_at,
    p.updated_at AS product_updated_at,
    p.deleted_at AS product_deleted_at,
    pvs.id AS variation_stock_id,
    pvs.stock,
    pvp.id AS variation_price_id,
    pvp.price,
    pv.id AS product_variation_id,
    pv.weight,
    pv.status AS variation_status
FROM 
    public.orders o
JOIN 
    public.product_ratings pr ON o.id = pr.order_id
JOIN 
    public.products p ON pr.product_id = p.id
JOIN 
    public.product_names pn ON p.name_id = pn.id
JOIN 
    public.categories c ON pn.category_id = c.id
JOIN 
    public.product_variations pv ON p.id = pv.product_id
JOIN 
    public.product_variation_stocks pvs ON pv.id = pvs.product_variation_id
JOIN 
    public.product_variation_prices pvp ON pv.id = pvp.product_variation_id;
"""

# Fetch data into a DataFrame
df = pd.read_sql(query, engine)

# View the first few rows
df.head()


,order_id,groups_carts_id,order_status,total_amount,order_created_at,order_updated_at,order_deleted_at,payment_method,discount,rating,...,product_created_at,product_updated_at,product_deleted_at,variation_stock_id,stock,variation_price_id,price,product_variation_id,weight,variation_status
0,8437e859-05e2-4bbd-adce-9a24cda8ed1e,f5bd5feb-7685-46ef-b305-79f6d0d34bfe,COMPLETED,1198.0,2024-09-08 12:57:02.073086,2024-09-08 12:58:31.096846,None,CHIPCHIP_PAY,0.0,5,...,2024-07-29 08:47:58.350028,2024-10-23 05:15:11.144202,None,dc66838c-7f95-4d33-863b-39047d5cf454,41,4a127a46-5974-4e2c-8486-1dab2215dfe7,599.0,56c30f82-9359-4837-a832-c8bbf561ea86,0.35,INACTIVE
1,8437e859-05e2-4bbd-adce-9a24cda8ed1e,f5bd5feb-7685-46ef-b305-79f6d0d34bfe,COMPLETED,1198.0,2024-09-08 12:57:02.073086,2024-09-08 12:58:31.096846,None,CHIPCHIP_PAY,0.0,5,...,2024-07-29 08:47:58.350028,2024-10-23 05:15:11.144202,None,dc66838c-7f95-4d33-863b-39047d5cf454,41,e2686d2f-56ab-4d02-8a09-d6d0cca800a7,599.0,56c30f82-9359-4837-a832-c8bbf561ea86,0.35,INACTIVE
2,8437e859-05e2-4bbd-adce-9a24cda8ed1e,f5bd5feb-7685-46ef-b305-79f6d0d34bfe,COMPLETED,1198.0,2024-09-08 12:57:02.073086,2024-09-08 12:58:31.096846,None,CHIPCHIP_PAY,0.0,5,...,2024-07-29 08:47:58.350028,2024-10-23 05:15:11.144202,None,dc66838c-7f95-4d33-863b-39047d5cf454,41,9faf7646-2753-4915-b50d-c1a121c661a7,599.0,56c30f82-9359-4837-a832-c8bbf561ea86,0.35,INACTIVE
3,8437e859-05e2-4bbd-adce-9a24cda8ed1e,f5bd5feb-7685-46ef-b305-79f6d0d34bfe,COMPLETED,1198.0,2024-09-08 12:57:02.073086,2024-09-08 12:58:31.096846,None,CHIPCHIP_PAY,0.0,5,...,2024-07-29 08:47:58.350028,2024-10-23 05:15:11.144202,None,dc66838c-7f95-4d33-863b-39047d5cf454,41,2396868d-a45b-4fd7-ade2-5b76d29a418f,599.0,56c30f82-9359-4837-a832-c8bbf561ea86,0.35,INACTIVE
4,8437e859-05e2-4bbd-adce-9a24cda8ed1e,f5bd5feb-7685-46ef-b305-79f6d0d34bfe,COMPLETED,1198.0,2024-09-08 12:57:02.073086,2024-09-08 12:58:31.096846,None,CHIPCHIP_PAY,0.0,5,...,2024-07-29 08:47:58.350028,2024-10-23 05:15:11.144202,None,dc66838c-7f95-4d33-863b-39047d5cf454,41,4d627ce2-b65f-424f-bc76-7ff396c3981b,599.0,56c30f82-9359-4837-a832-c8bbf561ea86,0.35,INACTIVE


Feature Engineering

In [5]:
# Convert order_created_at to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])

# Temporal features
df['day_of_week'] = df['order_created_at'].dt.dayofweek
df['week_of_year'] = df['order_created_at'].dt.isocalendar().week
df['month'] = df['order_created_at'].dt.month

# Aggregate data to weekly level
weekly_sales = df.groupby(['product_id', 'week_of_year']).agg({
    'total_amount': 'sum',
    'stock': 'mean',
    'price': 'mean',
    'rating': 'mean'
}).reset_index()

# Add target column: Label products likely to sell out
weekly_sales['likely_to_sell_out'] = (weekly_sales['stock'] <= 10).astype(int)

# View engineered data
weekly_sales.head()


,product_id,week_of_year,total_amount,stock,price,rating,likely_to_sell_out
0,29a43441-62c8-4731-82e4-9e4f94431f0b,33,756.0,57.166667,24.190476,5.0,0
1,84204980-a42d-4fb1-bdba-63048eae770b,40,756.0,20.000000,252.000000,5.0,0
2,9f0c5873-6021-4555-a46f-5e6a14d21a8c,33,215041.0,29.768802,556.381616,5.0,0
3,9f0c5873-6021-4555-a46f-5e6a14d21a8c,36,645123.0,29.768802,556.381616,5.0,0
4,c285cd24-538d-4e66-bdf7-6d18e1a6ea92,43,5085.0,100.000000,13.416667,5.0,0
